<a href="https://colab.research.google.com/github/maybachar/getpet-recognition/blob/master/train_model_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.models import model_from_json

# num of breeds and image size.
num_breeds = 120
size = 224

# Append to lables list the suitale lable for each image using info from csv file
# and add the number of breed instead the name breed(using dictionary).   
def create_lables_list(img_id, img_lables,labels_file, breed_num_dic):
    img_id = img_id.split("/")[-1]
    img_id = img_id.split(".")[0]
    # get breed name by image id from labels.csv
    breed_name = list(labels_file[labels_file.id == img_id]["breed"])[0]
    # get the number of this breed and append it to list.
    breed_idx = breed_num_dic[breed_name]
    img_lables.append(breed_idx)


def build():
    # The input size the model will get. default size for mobile and 3 is for colors.
    inputs_model = Input((size, size, 3))
    # Create the base pre-trained model.
    base_model = MobileNetV2(input_tensor=inputs_model, include_top=False,weights="imagenet", input_shape=(size, size, 3))
    # Set all parameters to be trainable
    base_model.trainable = True
    # Get the model so we can add layers to it.
    model_out = base_model.output
    # Add a global spatial average pooling layer
    model_out = GlobalAveragePooling2D()(model_out)
    # Include a dropout layer to minimize the overfitting.
    model_out = Dropout(0.2)(model_out)
    # Using activation function RELU- widely used in CNN.
    model_out = Dense(1024, activation="relu")(model_out)
    # Add softmax layer for getting probabilities on the breeds.
    model_out = Dense(num_breeds, activation="softmax")(model_out)
    # Create the model with it's layers.
    model = tf.keras.Model(inputs_model, model_out)
    return model


def fix_data_element(path, y):
    # one hot encoding to lable (all zero except for the place y).
    one_hot_y = [0] * num_breeds
    one_hot_y[y] = 1
    one_hot_y = np.array(one_hot_y)
    # Cast to type as defined.
    one_hot_y = one_hot_y.astype(np.int32)
    path = path.decode()
    # loads a color image from the specified file.
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    # change the width, height of an image.
    img = cv2.resize(img, (size, size))
    # normaliztion.
    img = img / 255.0
    img = img.astype(np.float32)
    return img, one_hot_y

def wrap_elements(x, y):
    # wrap numpy function as an operation in TensorFlow function.
    x, y = tf.numpy_function(fix_data_element, [x, y], [tf.float32, tf.int32])
    # resize image to the image size mobilenet expect to get.
    x.set_shape((size, size, 3))
    y.set_shape((num_breeds))
    return x, y

def fix_dataset(x, y, batch):
    # get the slices of x and y into one dataset.
    data_slices = tf.data.Dataset.from_tensor_slices((x, y))
    # wrap with fix_data_element function every element of the Dataset separately and resize.
    data_slices = data_slices.map(wrap_elements)
    # combines consecutive elements of a dataset object into batches.
    data_slices = data_slices.batch(batch)
    return data_slices


if __name__ == "__main__":
    # learning rate 0.0001.
    lr = 0.0001
    batch = 16
    epochs = 5
    # paths of train data folder and csv file with labels of each pic.
    path = "/content/drive/MyDrive/dog_breed_identification/"
    train_path = os.path.join(path, "train/*")
    labels_path = os.path.join(path, "labels.csv")
    # import the CSV file.
    labels_file = pd.read_csv(labels_path)
    # get all unique breeds in file.
    breed = labels_file["breed"].unique()
    length_breed=len(breed)
    breed_num_dic = {}
    # create dictionary of breed and number.
    for i in range(length_breed):
      breed_num_dic[breed[i]] = i
    # ids of photos
    all_img_ids = glob(train_path)
    # list of breed index by order of image id
    img_lables = []
    # adding the num of breed-the lable of each image to lables list.
    for image_id in all_img_ids:
        create_lables_list(image_id, img_lables,labels_file, breed_num_dic)
    # work only with N image id and N lables of the breed id which suitable to the images.
    all_img_ids = all_img_ids[:]
    img_lables = img_lables[:]
    # spliting data to train and validation.
    train_x, validation_x = train_test_split(all_img_ids, test_size=0.2, random_state=42)
    train_y, validation_y = train_test_split(img_lables, test_size=0.2, random_state=42)
    # dataset
    trainSet = fix_dataset(train_x, train_y, batch=batch)
    validationSet = fix_dataset(validation_x, validation_y, batch=batch)
    #create a CNN model with it's layers.
    model = build()
    #Configures the model for training. optimizer- adam.
    model.compile(loss="categorical_crossentropy", optimizer=Adam(lr), metrics=["acc"])
    # Train
    # set of functions to be applied at training procedure:
    # modelCheckPoint saves the model after every epoch.
    # Reduce learning rate when a metric has stopped improving.
    callbacks = [ModelCheckpoint("model.h5", verbose=1, save_best_only=True),
        ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.000001)]
    # Trains the model for number of epochs. evaluate the loss at the end of each epoch.
    model.fit(trainSet, validation_data=validationSet, epochs=epochs, callbacks=callbacks)
    # save the model and it's weights in H5 format to jeson file.
    model_json = model.to_json()
    with open("/content/drive/MyDrive/dog_breed_identification/model.json", "w") as json_file:
        json_file.write(model_json)
        model.save_weights("/content/drive/MyDrive/dog_breed_identification/model.h5")
        print("Saved model to disk")


Epoch 1/5
512/512 [==============================] - 2243s 4s/step - loss: 2.7838 - acc: 0.3358 - val_loss: 1.6946 - val_acc: 0.5237

Epoch 00001: val_loss improved from inf to 1.69456, saving model to model.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/5
512/512 [==============================] - 1654s 3s/step - loss: 0.9585 - acc: 0.7316 - val_loss: 1.3319 - val_acc: 0.6039

Epoch 00002: val_loss improved from 1.69456 to 1.33192, saving model to model.h5
Epoch 3/5
512/512 [==============================] - 1624s 3s/step - loss: 0.3390 - acc: 0.9188 - val_loss: 1.2531 - val_acc: 0.6499

Epoch 00003: val_loss improved from 1.33192 to 1.25305, saving model to model.h5
Epoch 4/5
512/512 [==============================] - 1625s 3s/step - loss: 0.1078 - acc: 0.9865 - val_loss: 1.1641 - val_acc: 0.6670

Epoch 00004: val_loss improved from 1.25305 to 1.16410, saving model to model.h5
Epoch 5/5
512/512 [==============================] - 1640s 3s/step - loss: 0.0411 - acc: 0.9967 - val_loss: 1.0942 - val_acc: 0.6841

Epoch 00005: val_loss improved from 1.16410 to 1.09423, saving model to model.h5
Saved model to disk
